In [1]:
import requests
import json
import time
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

access_token = "YmY_hVX5X-e9zKMBPCowCVlvIOpmdkq9wP8H4MEJptE"

query = """
query ($afterDate: DateTime!, $beforeDate: DateTime!, $cursor: String) {
  posts(postedAfter: $afterDate, postedBefore: $beforeDate, after: $cursor) {
    pageInfo {
      endCursor
      hasNextPage
    }
    edges {
      node {
        id
        name
        votesCount
        reviewsRating
        reviewsCount
        commentsCount
        votes {
          totalCount
        }
        createdAt
        collections {
          totalCount
        }
        topics {
          edges {
            node {
              name
              followersCount
              postsCount
            }
          }
        }
      }
    }
  }
}
"""

In [7]:
def send_graphql_request(query, access_token, variables=None):

  url = "https://api.producthunt.com/v2/api/graphql"

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}",
  }

  data = {"query": query, "variables": variables}

  response = requests.post(url, headers=headers, data=json.dumps(data))

  # Check rate limit headers
  limit = int(response.headers['X-Rate-Limit-Limit'])
  remaining = int(response.headers['X-Rate-Limit-Remaining'])
  reset = int(response.headers['X-Rate-Limit-Reset'])
  # print("limit: ", limit)
  print("remaining: ", remaining)
  print("reset: ", reset)

  def sleep_with_progress(seconds):
    for remaining in tqdm(range(seconds), desc="Sleeping", unit="sec", ascii=True):
        time.sleep(1)

  if remaining < 100:
    # print("Waiting...")
    # time.sleep(int(reset)) # l
    sleep_with_progress(int(reset))


  return response.json()


In [8]:

# Specify dates
after_date = "2023-03-01"
before_date = "2023-04-01"

# Initial variables
variables = {"afterDate": after_date, "beforeDate": before_date, "cursor": None}

all_posts = []


In [9]:
file_path = "/content/drive/My Drive/CS_Projects/2023/Posts_Mar_2023_Fix_02.json"

while True:

  response = send_graphql_request(query, access_token, variables)
  print("response", response)
  # print("response", type(response))
  # Check response format
  if "data" not in response or "posts" not in response["data"]:
    print("Invalid response format.")
    break

  posts = response["data"]["posts"]["edges"]
  all_posts.extend(posts)

  with open(file_path, "a") as file:
    json.dump(all_posts, file)

  pageInfo = response["data"]["posts"]["pageInfo"]
  has_next_page = pageInfo["hasNextPage"]

  if has_next_page:
    cursor = pageInfo["endCursor"]
    variables["cursor"] = cursor

  else:
    break

  print("Saved!")



{'errors': [{'message': "Field 'reviews' doesn't exist on type 'Post'", 'locations': [{'line': 15, 'column': 9}], 'path': ['query', 'posts', 'edges', 'node', 'reviews'], 'extensions': {'code': 'undefinedField', 'typeName': 'Post', 'fieldName': 'reviews'}}]}
Invalid response format.
